# 拆分数据集

In [1]:
import cv2
import os
import keras
from dog_generator import CDGenerator
from sklearn.cross_validation import train_test_split

def load_data(path):
    img_names=[]
    img_labels=[]
    for dpath,dnames,fnames in os.walk(path):
#         img_names=fnames
#         print(fnames)
        for i in fnames:
            img_names.append("train/" + i)
            img_labels.append(i[:3])
#     print(img_names[-20:])
#     print(img_labels[-20:])
    train_x,valid_y,labels_x,labels_y = train_test_split(img_names,img_labels,test_size=0.3,random_state=42)

#     cate_dict={"cat":0,"dog":1}
#     print(labels_x[:30])
#     print(labels_y[:30])
#     print(len(labels_x))
    train_labels=[]
    valid_labels=[]
    for i in labels_x:
        if i == 'cat':
            train_labels.append(0)
        else:
            train_labels.append(1)
       
    for j in labels_y:
        if j == 'cat':
            valid_labels.append(0)
        else:
            valid_labels.append(1)
#     print(len(train_labels),len(valid_labels))
#     print(valid_labels[:30])

    return train_x,valid_y,train_labels,valid_labels
# load_data("train")

C:\Users\15547\Anaconda3\envs\py3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\15547\Anaconda3\envs\py3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#加载测试集图片
import cv2
def load_test():
    imgs = []
    for i in os.listdir('test'):
#         img = cv2.imread('test/'+i)
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#         img = cv2.resize(img,des_size)
#         imgs.append(img)
        
        imgs.append('test/'+i)
#     print(len(imgs))
    return imgs


# Generator

In [3]:
train_img,valid_img,train_labels,valid_labels=load_data("train")
batch_size = 32
epochs = 5
# print(len(train_labels))
train_generator = CDGenerator((train_img,train_labels),
                                len(train_img),'train',is_directory=True,batch_size=batch_size,seed=0)

valid_generator = CDGenerator((valid_img,valid_labels),
                                len(valid_img),'valid',is_directory=True,batch_size=batch_size,seed=0)

test_imgs = load_test()
test_generator = CDGenerator((test_imgs,None),
                                len(test_imgs),'test',is_directory=True,batch_size=batch_size,seed=0)
# print(test_generator)

In [15]:
train_img,valid_img,train_labels,valid_labels=load_data("train")
batch_size = 32
epochs = 5

train_generator = CDGenerator((train_img,train_labels),
                                len(train_img),'train',des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

valid_generator = CDGenerator((valid_img,valid_labels),
                                len(valid_img),'valid',des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

test_imgs = load_test()
test_generator = CDGenerator((test_imgs,None),len(test_imgs),'test',
                             des_size=(299,299),is_directory=True,
                             batch_size=batch_size,seed=0)

# 导入库

In [4]:
from keras.applications import vgg19,resnet50,inception_v3,xception
from keras.models import Sequential,Model
from keras.layers import Input,Convolution2D,MaxPooling2D,Dense,Activation,Dropout,Flatten
import h5py
from keras.applications.resnet50 import preprocess_input as res_preprocess_input
from keras.applications.xception import preprocess_input as xcep_preprocess_input
from keras.applications.inception_v3 import preprocess_input as incep_preprocess_input
import numpy as np
from keras.applications.imagenet_utils import decode_predictions

# 构建网络

In [8]:
Input = keras.layers.Input(shape=(224,224,3))
process_input = keras.layers.Lambda(res_preprocess_input)(Input)
resnet = resnet50.ResNet50(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
resModel = Model(inputs=resnet.input,outputs=resnet.output)
output = resModel.output
output=Dropout(0.5)(output)
predictions = Dense(1,activation="sigmoid")(output)
model = Model(inputs=resModel.input, outputs=predictions)

for layers in resnet.layers:
    layers.trainable=False
    
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,len(train_img)//batch_size,
                    epochs=epochs,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_img)// batch_size)

Epoch 1/5
546/546 [==============================] - 220s 402ms/step - loss: 0.1426 - acc: 0.9405 - val_loss: 0.0618 - val_acc: 0.9736
Epoch 2/5
546/546 [==============================] - 216s 395ms/step - loss: 0.0821 - acc: 0.9697 - val_loss: 0.0470 - val_acc: 0.9814
Epoch 3/5
546/546 [==============================] - 215s 395ms/step - loss: 0.0748 - acc: 0.9714 - val_loss: 0.0645 - val_acc: 0.9752
Epoch 4/5
546/546 [==============================] - 214s 392ms/step - loss: 0.0778 - acc: 0.9712 - val_loss: 0.0499 - val_acc: 0.9819
Epoch 5/5
546/546 [==============================] - 212s 388ms/step - loss: 0.0722 - acc: 0.9720 - val_loss: 0.0381 - val_acc: 0.9867


In [ ]:
model.save_weights('resnet_weights.h5')

In [ ]:
train_img,valid_img,train_labels,valid_labels=load_data("train")
batch_size = 32
epochs = 5
# print(len(train_labels))
train_generator = CDGenerator((train_img,train_labels),
                                len(train_img),des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

valid_generator = CDGenerator((valid_img,valid_labels),
                                len(valid_img),des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

In [13]:
Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(incep_preprocess_input)(Input)
ince_v3 = inception_v3.InceptionV3(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
inceModel = Model(inputs=ince_v3.input,outputs=ince_v3.output)
output = inceModel.output
output=Dropout(0.5)(output)
predictions = Dense(1,activation="sigmoid")(output)
model = Model(inputs=inceModel.input, outputs=predictions)

for layers in ince_v3.layers:
    layers.trainable=False
    
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,len(train_img)//batch_size,
                    epochs=epochs,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_img)// batch_size)

Epoch 1/5
546/546 [==============================] - 295s 540ms/step - loss: 0.1866 - acc: 0.9315 - val_loss: 0.0666 - val_acc: 0.9812
Epoch 2/5
546/546 [==============================] - 284s 521ms/step - loss: 0.1103 - acc: 0.9592 - val_loss: 0.0428 - val_acc: 0.9884
Epoch 3/5
546/546 [==============================] - 283s 519ms/step - loss: 0.0973 - acc: 0.9639 - val_loss: 0.0545 - val_acc: 0.9850
Epoch 4/5
546/546 [==============================] - 284s 520ms/step - loss: 0.0968 - acc: 0.9656 - val_loss: 0.0656 - val_acc: 0.9807
Epoch 5/5
546/546 [==============================] - 284s 520ms/step - loss: 0.0924 - acc: 0.9651 - val_loss: 0.0707 - val_acc: 0.9791


In [ ]:
model.save_weights('ince_v3_weights.h5')

In [ ]:
train_img,valid_img,train_labels,valid_labels=load_data("train")
batch_size = 32
epochs = 5
# print(len(train_labels))
train_generator = CDGenerator((train_img,train_labels),
                                len(train_img),des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

valid_generator = CDGenerator((valid_img,valid_labels),
                                len(valid_img),des_size=(299,299),
                                is_directory=True,batch_size=batch_size,seed=0)

In [23]:
Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(xcep_preprocess_input)(Input)
xception = xception.Xception(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
xceModel = Model(inputs=xception.input,outputs=xception.output)
output = xceModel.output
output=Dropout(0.25)(output)
predictions = Dense(1,activation="sigmoid")(output)
model = Model(inputs=xceModel.input, outputs=predictions)

for layers in xception.layers:
    layers.trainable=False
    
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,len(train_img)//batch_size,
                    epochs=epochs,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_img)// batch_size)

Epoch 1/5
546/546 [==============================] - 515s 943ms/step - loss: 0.1148 - acc: 0.9720 - val_loss: 0.0893 - val_acc: 0.9782
Epoch 2/5
546/546 [==============================] - 512s 937ms/step - loss: 0.0547 - acc: 0.9821 - val_loss: 0.0563 - val_acc: 0.9863
Epoch 3/5
546/546 [==============================] - 511s 937ms/step - loss: 0.0465 - acc: 0.9848 - val_loss: 0.0726 - val_acc: 0.9804
Epoch 4/5
546/546 [==============================] - 512s 937ms/step - loss: 0.0417 - acc: 0.9861 - val_loss: 0.0699 - val_acc: 0.9813
Epoch 5/5
546/546 [==============================] - 512s 937ms/step - loss: 0.0388 - acc: 0.9860 - val_loss: 0.0713 - val_acc: 0.9807


In [ ]:
model.save_weights('xception_weights.h5')

Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(xcep_preprocess_input)(Input)
xception = xception.Xception(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
xceModel = Model(inputs=xception.input,outputs=xception.output)
output = xceModel.output
output=Dropout(0.25)(output)
predictions = Dense(1,activation="sigmoid")(output)
model = Model(inputs=xceModel.input, outputs=predictions)

for layers in xception.layers:
    layers.trainable=False
    
model.load_weights('xception_weights.h5')

test_imgs = load_test((299,299))
# print(process_input.shape)
# print(test_imgs[0].shape)
image = test_imgs[0]
new_img = np.expand_dims(image,axis=0)
print(new_img.shape)
result = model.predict(new_img)
print(result)
print(decode_predictions(result,top=1000))

# 特征向量的导出

In [5]:
import h5py

def output_features(model_name,model,train_generator,train_labels,valid_generator,valid_labels,test_generator,batch_size=32):
    if model_name == 'resnet_features':
        model.load_weights('resnet_weights.h5',by_name=True)
    elif model_name == 'ince_v3_features':
        model.load_weights('ince_v3_weights.h5',by_name=True)
    else:
        model.load_weights('xception_weights.h5',by_name=True)
    
    #将labels转成array
    train_labels = np.array(train_labels)
    valid_labels = np.array(valid_labels)
    
    #trian、valid、test的features
    train_features = model.predict_generator(train_generator,int(np.ceil(train_generator.n/batch_size)),verbose=1)
    valid_features = model.predict_generator(valid_generator,int(np.ceil(valid_generator.n/batch_size)),verbose=1)
    test_features = model.predict_generator(test_generator,int(np.ceil(test_generator.n/batch_size)),verbose=1)
    
    with h5py.File(model_name+'.h5','w') as file:
        file.create_dataset('train',data=train_features,dtype='float32')
        file.create_dataset('train_labels',data=np.array(train_generator.y),dtype='uint8')
        file.create_dataset('valid',data=valid_features,dtype='float32')
        file.create_dataset('valid_labels',data=np.array(valid_generator.y),dtype='uint8')
        file.create_dataset('test',data=test_features,dtype='float32')
        
        

In [7]:
#resnet
res50 = resnet50.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")
print(res50.output)
output_features('resnet_features',Model(inputs=res50.input,outputs=res50.output),
                train_generator,train_labels,valid_generator,valid_labels,test_generator)

Tensor("global_average_pooling2d_2/Mean:0", shape=(?, 2048), dtype=float32)


StopIteration: 'str' object cannot be interpreted as an integer

In [18]:
#inception
Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(incep_preprocess_input)(Input)
ince_v3 = inception_v3.InceptionV3(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
print(ince_v3.output)
output_features('ince_v3_features',Model(inputs=ince_v3.input,outputs=ince_v3.output),
                train_generator,train_labels,valid_generator,valid_labels,test_generator)

Tensor("global_average_pooling2d_3/Mean:0", shape=(?, 2048), dtype=float32)
391/391 [==============================] - 169s 433ms/step


In [21]:
#xception
Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(xcep_preprocess_input)(Input)
xception = xception.Xception(include_top=False,weights="imagenet",input_tensor=process_input,pooling="avg")
print(xception.output)
output_features('xception_features',Model(inputs=xception.input,outputs=xception.output),
                train_generator,train_labels,valid_generator,valid_labels,test_generator)

Tensor("global_average_pooling2d_8/Mean:0", shape=(?, 2048), dtype=float32)
391/391 [==============================] - 310s 793ms/step


# 模型的融合

In [4]:
feature_files = ['resnet_features.h5','ince_v3_features.h5','xception_features.h5']

X_train =[]
y_train =[]
X_valid =[]
y_valid =[]
X_test =[]

for file_name in feature_files:
    with h5py.File(file_name,'r') as h:
        X_train.append(np.array(h['train']))
        y_train = (np.array(h['train_labels']))
        
        X_test.append(np.array(h['test']))
        
        X_valid.append(np.array(h['valid']))
        y_valid = (np.array(h['valid_labels']))
        print(np.array(h['train']).shape,np.array(h['valid']).shape,np.array(h['test']).shape)
#         print(X_train[:3],y_train[:3])
train = np.concatenate(X_train,axis=1)
valid = np.concatenate(X_valid,axis=1)
test  = np.concatenate(X_test, axis=1)
print(train.shape,valid.shape,test.shape)


(17500, 2048) (7500, 2048) (12500, 2048)
(17500, 2048) (7500, 2048) (12500, 2048)
(17500, 2048) (7500, 2048) (12500, 2048)
(17500, 6144) (7500, 6144) (12500, 6144)


In [5]:
from sklearn.utils import shuffle
# print(np.array(y_train).shape)
X_train,y_train = shuffle(train,y_train)
# print(len(X_train),len(y_train))

# Train

In [7]:
import keras.utils

input_tensor = Input(X_train.shape[1:])
print(input_tensor)
# print(X_train)
x = Dropout(0.5)(input_tensor)
output = Dense(1,activation='sigmoid')(x)

con_model = Model(inputs=input_tensor,outputs=output)
con_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

con_model.fit(X_train,y_train,batch_size=128, epochs=5,validation_data=(valid,y_valid))

Tensor("input_2:0", shape=(?, 6144), dtype=float32)
Train on 17500 samples, validate on 7500 samples
Epoch 1/5
17500/17500 [==============================] - 1s 80us/step - loss: 0.7712 - acc: 0.4906 - val_loss: 0.7136 - val_acc: 0.5021
Epoch 2/5
17500/17500 [==============================] - 1s 71us/step - loss: 0.7345 - acc: 0.5211 - val_loss: 0.7056 - val_acc: 0.5073
Epoch 3/5
17500/17500 [==============================] - 1s 74us/step - loss: 0.7254 - acc: 0.5252 - val_loss: 0.7093 - val_acc: 0.5023
Epoch 4/5
17500/17500 [==============================] - 1s 76us/step - loss: 0.7129 - acc: 0.5392 - val_loss: 0.7167 - val_acc: 0.5012
Epoch 5/5
17500/17500 [==============================] - 1s 74us/step - loss: 0.7108 - acc: 0.5448 - val_loss: 0.7098 - val_acc: 0.5081
